In [0]:
import os
import re
import gzip
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path
import shutil

# Ruta origen (DBFS local path)
source_dir = "/dbfs/FileStore/pubmed_baseline"
target_dir = "/dbfs/FileStore/pubmed_filtrado"
Path(target_dir).mkdir(parents=True, exist_ok=True)

# Compilar expresión regular para años
year_pattern = re.compile(r"\b(2020|2021|2022|2023|2024|2025)\b")

# Función para buscar año en el fichero
def contiene_anio(file_path):
    try:
        with gzip.open(file_path, 'rt', encoding='utf-8', errors='ignore') as f:
            for i, line in enumerate(f):
                if year_pattern.search(line):
                    # Copiar a destino
                    dest_path = os.path.join(target_dir, os.path.basename(file_path))
                    shutil.copy(file_path, dest_path)
                    return f"✅ {os.path.basename(file_path)} copiado"
                if i > 5000:  # Evitar leer archivos completos si no hace falta
                    break
        return f"⏭️ {os.path.basename(file_path)} ignorado"
    except Exception as e:
        return f"❌ Error {os.path.basename(file_path)}: {e}"

# Listar archivos
archivos = [os.path.join(source_dir, f.name) for f in dbutils.fs.ls("dbfs:/FileStore/pubmed_baseline") if f.name.endswith(".xml.gz")]

# Procesar en paralelo
with ThreadPoolExecutor(max_workers=32) as executor:
    results = list(executor.map(contiene_anio, archivos))

# Mostrar resumen
for r in results:
    print(r)

In [0]:
from pyspark.sql.functions import input_file_name


files = dbutils.fs.ls("dbfs:/FileStore/pubmed_filtrado")

num_files = len(files)
total_size = sum(f.size for f in files)

print(f"Número de ficheros: {num_files}")
print(f"Tamaño total: {total_size/1024/1024:.2f} MB")